## Load document 

In [24]:
from langchain_community.document_loaders import TextLoader

doc_path = "/Users/shakibibnashameem/Documents/Practice/LangChain/rag/odessy.txt"

loader = TextLoader(file_path=doc_path)
doc = loader.load()

## Split into chunks

In [25]:
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter(chunk_size = 2500, chunk_overlap = 100)
chunks = splitter.split_documents(doc)

## Embeddings

In [26]:
from langchain_ollama import OllamaEmbeddings

embedding = OllamaEmbeddings(
    model='nomic-embed-text'
)

## Setup Vector DB

In [27]:
from langchain_community.vectorstores import Chroma

vector_db = Chroma.from_documents(
    documents = chunks,
    embedding=embedding
)

## Setup Prompt

In [32]:
from langchain.prompts import ChatPromptTemplate

prompt_template = """
        You are an helpful AI assistant. Answer the question below based on the the context, If you 
        don't know the answer, replay that I don't know the answer. Don't provide any extra information. 

        Context : {context}
        Question : {question}
"""
prompt = ChatPromptTemplate.from_template(prompt_template)

## Retriever

In [38]:
retriever = vector_db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 5, "score_threshold": 0.3},
    prompt = prompt
)

In [39]:
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


model = ChatOllama(model='llama3.1', seed = 42)

chain = (
    {'context' : retriever, 'question' : RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

res = chain.invoke("Who is Odysseus' wife?")

print(res)

According to the provided text, Odysseus' wife is Penelope.
